<a href="https://colab.research.google.com/github/zephyris/scarlesstagging/blob/main/examples/ipynb/ScarlessTagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#@title Usage instructions

#@markdown Enter your gene IDs and choose the tagging terminus, then select `Runtime > Run all` or press `Ctrl+F9` to run all of the code.
#@markdown To do further primer design, you only need to re-run the "User settings" and "Find tagging primers" sections.
!pip install -q git+https://github.com/zephyris/scarlesstagging

In [6]:
#@title User settings

#@markdown List of gene IDs for which to design primers
# as in TriTrypDB
gene_ids = "Tb927.7.6580 Tb427.07.6580 Tb427_070071900 " #@param {type:"string"}
gene_ids = gene_ids.split()

#@markdown Tagging terminus
# "n", "c" or "both"
terminus = "c" #@param string ["n", "c", "both"]

#@markdown Plasmid system/design strategy
#@markdown pPOT-compatible gives pRExT2A-compatible primers which can also be used with pPOT plasmids, but leave a small scar
#@markdown pRExT2A-scarless gives primers for scarless integration, but may be drug/tag-specific and incompatible with pPOT plasmids
# defines primer binding site sequences
plasmid_system = "ppot-compatible" #@param string ["ppot-compatible", "prext2a-scarless"]

#@markdown Tag and drug marker (when using 2A plasmid system)
tag = "mng"  #@param string ["msc", "mng"]
drug = "bsr" #@param string ["pac", "bsr"]

In [7]:
#@title Find tagging primers

#@markdown Exhaustively scans the start (for N terminal tagging) or end (for C terminal tagging) of the open reading frame for PAM sites which can be mutated to remove the PAM without changing the coding sequence.

#@markdown Up to six types of primers are designed:

from scarlesstagging import ScarlessTagging

scarlesstagging = ScarlessTagging()
primer_results = []
for gene_id in gene_ids:
    primer_results.append(scarlesstagging.design_primers(gene_id, terminus, plasmid_system, tag, drug))

file = open("primers.tsv", "w")
file.write("\t".join(["Gene ID", "UF", "UR", "USG", "UERR", "DF", "DR", "DSG", "DERR"])+"\r\n")
for result in primer_results:
  for err in ["uerr", "derr"]:
    if result[err] is None:
      result[err] = "None"
    else:
      result[err] = ", ".join(result[err])
  for key in result:
    print("\t".join([str(x) for x in [key, result[key]]]))
  print("")
  file.write("\t".join([str(x) for x in [result["id"], result["uf"], result["ur"], result["usg"], result["uerr"], result["df"], result["dr"], result["dsg"], result["derr"]]])+"\r\n")
file.close()

from google.colab import files
files.download("primers.tsv")

#@markdown Sequences which are shared among all primers of that type (ie. standard plasmid annealing sequences, sgRNA T7 promoter sequence) are shown in lower case.
#@markdown Gene/terminus-specific sequences (homology arms and actual guide RNA) are shown in upper case.
#@markdown The single nucleotide change to mutate the PAM site is shown in lowercase, in the upstream reverse primer for N terminal tagging, and the downstream reverse primer for C terminal tagging.

#@markdown Make sure to check the upstream error (UERR) and downstream error (DERR) columns for any errors or warnings about the primer design.